In [23]:
import qiskit
from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator
import numpy as np

# --- 1. Helper Function: The Quantum Radar Circuit ---

def create_ship_scanner(is_ship):
    """
    Creates a 2-qubit "Quantum Radar" circuit based on a
    Mach-Zehnder interferometer.
    
    q0: The "Probe" qubit (our photon)
    q1: The "Target" qubit (the grid square)
    c0: The classical bit to store our radar's result
    """
    qc = QuantumCircuit(2, 1)

    # This section acts as the oracle. We set the target
    # qubit (q1) to |1> only if a ship is present.
    if is_ship:
        qc.x(1)  # A ship is present, so flip the target to |1>

    # Building the interferometer (H-CZ-H) 
    
    # H: Put probe (q0) in superposition. 
    qc.h(0)
    
    # CZ: If a ship is present (q1=|1>), it flips the phase of the probe.
    # If q1=|0> (water), it does nothing.
    qc.cz(0, 1)
    
    # 3. Recombine the probe's paths.
    # If the phase was flipped (ship), we get destructive interference -> |1>
    # If not (water), we get constructive interference -> |0>
    qc.h(0)

    # We measure the probe (q0), to see the interference result.
    qc.measure(0, 0)
    
    return qc

# Map of the battlefield
# 0 = Water, 1 = Ship
BATTLE_GRID = [
    [0, 1, 0],  # A Ship at A2
    [0, 1, 0],  # A Ship at B2
    [0, 0, 1]   # A Ship at C3
]

grid_flat = np.array(BATTLE_GRID).flatten() # Make the 2D grid a simple list for easier enumeration
radar_map_flat = []                         # This will store our scan results
backend = AerSimulator()                    # Set up the simulator

print("🛰️  Quantum Radar activated. Scanning enemy grid...")
print("-------------------------------------------------")
print(f"Enemy Grid (Truth):\n{np.array(BATTLE_GRID)}\n")

# For each square, a new quantum circuit is created and executed
for i, square in enumerate(grid_flat):
    
    # Checking if there is a ship
    ship_present = (square == 1)
    
    # Build the specific quantum radar circuit for this square
    qc = create_ship_scanner(ship_present)
    
    # Run the circuit. We only need 1 shot because the
    # result is deterministic (it's not random).
    job = backend.run(qc, shots=1) 
    
    result = job.result()
    counts = result.get_counts(qc)
    
    # Extract the result ('0' for water, '1' for ship)
    ping_result = int(list(counts.keys())[0])
    radar_map_flat.append(ping_result)

# Reshape 1D list of results back into a 2D grid
radar_map = np.array(radar_map_flat).reshape(3, 3)

print(f"Quantum Radar Map (Results):\n{radar_map}\n")
print("-------------------------------------------------")

# Compare the quantum-generated map to the original grid
if np.array_equal(BATTLE_GRID, radar_map):
    print("All ships located!")
else:
    print("Something is wrong...")

🛰️  Quantum Radar activated. Scanning enemy grid...
-------------------------------------------------
Enemy Grid (Truth):
[[0 1 0]
 [0 1 0]
 [0 0 1]]

Quantum Radar Map (Results):
[[0 1 0]
 [0 1 0]
 [0 0 1]]

-------------------------------------------------
All ships located!
